## **Importing necessary package**

In [ ]:
import pandas as pd
from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# **1. Data Exploratory:**



## **1-a. Variables/ Data fields**

In [ ]:
Image('/content/drive/MyDrive/capstone1/All_Airlines/feature image 1.jpg')

In [ ]:
Image('/content/drive/MyDrive/capstone1/All_Airlines/feature image 2.jpg')

## **1-b. reviewing the dataset**

In [ ]:
# retreving data

df = pd.read_csv('all_airlines.csv')
df.head().T

In [ ]:
df.shape

# **6. Baseline Model:**

In [ ]:
df_new = df.copy()

In [ ]:
# Proceeding with the sampled data for model training and prediction
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

# Step 1: Feature Engineering
# Selecting relevant features for regression
features = [
    "Carrier Code", "Flight Number", "Origin Airport", "Destination Airport",
    "Scheduled elapsed time (Minutes)", "Day", "Month", "Year", 'Delay Carrier (Minutes)', 'Taxi-Out time (Minutes)', 'Taxi-In time (Minutes)',
    'Departure delay (Minutes)', 'Delay Weather (Minutes)',
    'Delay National Aviation System (Minutes)', 'Delay Security (Minutes)',
    'Delay Late Aircraft Arrival (Minutes)', 'Actual Elapsed Time (Minutes)'
]
target = "Arrival Delay (Minutes)"

# Encode categorical features
label_encoders = {}
for col in ["Carrier Code", "Origin Airport", "Destination Airport"]:
    le = LabelEncoder()
    df_new[col] = le.fit_transform(df_new[col])
    label_encoders[col] = le

# Splitting the dataset into train and test sets
X = df_new[features]
y = df_new[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Train a Regression Model
regressor = RandomForestRegressor(random_state=42, n_estimators=100)
regressor.fit(X_train, y_train)

# Step 3: Evaluate the Model
y_pred = regressor.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

evaluation_results = {
    "Mean Absolute Error (MAE)": mae,
    "Mean Squared Error (MSE)": mse,
    "R^2 Score": r2
}

evaluation_results

# **Feature Selection**

### Embedded

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Assuming 'regressor' and 'features' are already defined from the previous code

# Get feature importances
importances = regressor.feature_importances_

# Create a DataFrame for visualization
feature_importances = pd.DataFrame({'Feature': features, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Plotting feature importances
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importances)
plt.title('Feature Importance')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

# Print feature importances in descending order
print("Feature Importances:")
feature_importances

### Filter

In [ ]:
# Select only numeric columns
numeric_columns = df_new.select_dtypes(include=['float64', 'int64', 'int32'])

# Compute the correlation matrix for numeric columns
correlation_matrix = numeric_columns.corr()

# Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)
plt.title("Correlation Matrix for All Numeric Variables")
plt.show()

# Focus on the correlation with "Arrival Delay (Minutes)"
arrival_delay_corr = correlation_matrix['Arrival Delay (Minutes)'].sort_values(ascending=False)
arrival_delay_corr

### Wrapper

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Define feature columns
features = ["Day", "Month", "Year",'Carrier Code',
       'Origin Airport', 'Destination Airport',
       'Departure delay (Minutes)',
       'Taxi-Out time (Minutes)', 'Delay Carrier (Minutes)',
       'Delay Weather (Minutes)', 'Delay National Aviation System (Minutes)',
       'Delay Security (Minutes)', 'Delay Late Aircraft Arrival (Minutes)',
       'Scheduled Elapsed Time (Minutes)', 'Actual Elapsed Time (Minutes)',
       'Taxi-In time (Minutes)']
target = "Arrival Delay (Minutes)"

# Splitting the dataset
X = df_new[features]
y = df_new[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# List of categorical columns
categorical_cols = ["Carrier Code", "Origin Airport", "Destination Airport"]

# Apply Label Encoding for Carrier Code (Binary)
label_encoders = {}
le = LabelEncoder()
X_train["Carrier Code"] = le.fit_transform(X_train["Carrier Code"])
X_test["Carrier Code"] = le.transform(X_test["Carrier Code"])
label_encoders["Carrier Code"] = le

# Apply Frequency Encoding for Origin and Destination Airports
for col in ["Origin Airport", "Destination Airport"]:
    freq_map = X_train[col].value_counts().to_dict()  # Compute frequency
    X_train[col] = X_train[col].map(freq_map)
    X_test[col] = X_test[col].map(freq_map)  # Use same mapping for test data




# Initialize Linear Regression Model
model = LinearRegression()

# Apply Recursive Feature Elimination (RFE) to select the top 10 features
rfe = RFE(model, n_features_to_select=10)
X_train_rfe = rfe.fit_transform(X_train, y_train)
X_test_rfe = rfe.transform(X_test)

# Train the model with selected features
model.fit(X_train_rfe, y_train)

# Model Predictions
y_pred = model.predict(X_test_rfe)

# Evaluate Performance
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Extract selected feature names and coefficients
selected_features = np.array(X.columns)[rfe.support_]
feature_coefficients = model.coef_

# Create a DataFrame to display selected features with their coefficients
feature_importance_df = pd.DataFrame({
    "Feature": selected_features,
    "Coefficient": feature_coefficients
}).sort_values(by="Coefficient", ascending=False)

# Display evaluation metrics and feature importance
evaluation_results = {
    "Selected Features": selected_features.tolist(),
    "Mean Absolute Error (MAE)": mae,
    "Mean Squared Error (MSE)": mse,
    "R^2 Score": r2
}

# Show the evaluation results
print(evaluation_results)

# Display feature coefficients
print("\nFeature Coefficients:")
print(feature_importance_df)

# **Final Model after Feature Selection**

### RandomForest

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

features = ["Day", "Month", "Year", "Taxi-Out time (Minutes)", "Taxi-In time (Minutes)",
            "Origin Airport", "Carrier Code", "Destination Airport",
            "Scheduled Elapsed Time (Minutes)",
            "Actual Elapsed Time (Minutes)",
            "Departure delay (Minutes)",
            "Scheduled departure hour"]
target = "Arrival Delay (Minutes)"

# Encode categorical features
label_encoders = {}
for col in ["Carrier Code", "Origin Airport", "Destination Airport"]:
    le = LabelEncoder()
    df_new[col] = le.fit_transform(df_new[col])
    label_encoders[col] = le

# Splitting the dataset into train and test sets
X = df_new[features]
y = df_new[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Train the Random Forest Model
regressor = RandomForestRegressor(
    random_state=42,
    n_estimators=100,  # Increase trees for better learning
    max_depth=15,  # Allow deeper trees for more complex patterns
    min_samples_split=4,  # Slightly reduce min samples per split
    min_samples_leaf=2  # Slightly reduce min samples per leaf
)
regressor.fit(X_train, y_train)

# Evaluate the Model
y_pred = regressor.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

evaluation_results = {
    "Mean Absolute Error (MAE)": mae,
    "Mean Squared Error (MSE)": mse,
    "R^2 Score": r2
}

evaluation_results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

residuals = y_test - y_pred

plt.figure(figsize=(8,5))
sns.scatterplot(x=y_pred, y=residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.title("Residual Plot")
plt.show()

In [ ]:
y_train_pred = regressor.predict(X_train)

train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_pred)

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Train MAE: {train_mae:.2f}, Test MAE: {test_mae:.2f}")
print(f"Train RMSE: {train_rmse:.2f}, Test RMSE: {test_rmse:.2f}")

In [ ]:
train_r2 = regressor.score(X_train, y_train)
test_r2 = regressor.score(X_test, y_test)

print(f"Training R²: {train_r2:.4f}")
print(f"Testing R²: {test_r2:.4f}")


### XGBoost with Selected Features

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

# Ensure all categorical columns are encoded
categorical_features = ["Origin Airport", "Carrier Code", "Destination Airport"]

# Encode categorical variables
label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    df_new[col] = le.fit_transform(df_new[col])
    label_encoders[col] = le  # Store encoders in case you need them later


# Define features and target
features = ["Day", "Month", "Year", "Taxi-Out time (Minutes)", "Taxi-In time (Minutes)",
            "Origin Airport", "Carrier Code", "Destination Airport",
            "Scheduled Elapsed Time (Minutes)",
            "Scheduled departure hour",
            "Departure delay (Minutes)", "Actual Elapsed Time (Minutes)"]
target = "Arrival Delay (Minutes)"

# Splitting the dataset into train and test sets
X = df_new[features]
y = df_new[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the XGBoost Model
regressor = XGBRegressor(
    random_state=42,
    n_estimators=1200,  # Keep high for stability
    max_depth=10,  # Slightly reduce tree complexity
    learning_rate=0.01,  # Lower learning rate for fine-tuned updates
    reg_lambda=7,  # Moderate L2 regularization
    reg_alpha=4,  # Stronger L1 regularization to remove noise
    gamma=4,  # Prune unnecessary splits
    subsample=0.85,  # Ensure generalization
    colsample_bytree=0.9  # Use 90% of features per tree
)

# Train the model
regressor.fit(X_train, y_train)

# Evaluate the Model
y_pred = regressor.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

evaluation_results = {
    "Mean Absolute Error (MAE)": mae,
    "Mean Squared Error (MSE)": mse,
    "R^2 Score": r2
}

evaluation_results

In [ ]:
import pandas as pd
import numpy as np
import joblib
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

# Ensure all categorical columns are encoded
categorical_features = ["Origin Airport", "Carrier Code", "Destination Airport"]

# Encode categorical variables
label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    df_new[col] = le.fit_transform(df_new[col])
    label_encoders[col] = le  # Store encoders in case you need them later

# Define features and target
features = ["Day", "Month", "Year", "Taxi-Out time (Minutes)", "Taxi-In time (Minutes)",
            "Origin Airport", "Carrier Code", "Destination Airport",
            "Scheduled Elapsed Time (Minutes)", "Scheduled departure hour",
            "Departure delay (Minutes)", "Actual Elapsed Time (Minutes)"]
target = "Arrival Delay (Minutes)"

# Splitting the dataset into train and test sets
X = df_new[features]
y = df_new[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the XGBoost Model
regressor = XGBRegressor(
    random_state=42,
    n_estimators=1200,  # Keep high for stability
    max_depth=20,  # Slightly reduce tree complexity
    learning_rate=0.02,  # Lower learning rate for fine-tuned updates
    reg_lambda=7,  # Moderate L2 regularization
    reg_alpha=4,  # Stronger L1 regularization to remove noise
    gamma=4,  # Prune unnecessary splits
    subsample=0.85,  # Ensure generalization
    colsample_bytree=0.9  # Use 90% of features per tree
)

# Train the model
regressor.fit(X_train, y_train)

# Save the trained model
joblib.dump(regressor, "best_XGB_Regression_model.pkl")

# Evaluate the Model
y_pred = regressor.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

evaluation_results = {
    "Mean Absolute Error (MAE)": mae,
    "Mean Squared Error (MSE)": mse,
    "R^2 Score": r2
}

evaluation_results

In [ ]:
y_train_pred = regressor.predict(X_train)

train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_pred)

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Train MAE: {train_mae:.2f}, Test MAE: {test_mae:.2f}")
print(f"Train RMSE: {train_rmse:.2f}, Test RMSE: {test_rmse:.2f}")

In [ ]:
train_r2 = regressor.score(X_train, y_train)
test_r2 = regressor.score(X_test, y_test)

print(f"Training R²: {train_r2:.4f}")
print(f"Testing R²: {test_r2:.4f}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Compute correlation
correlation = df_new[['Arrival Delay (Minutes)', 'Departure delay (Minutes)']].corr()
print(correlation)

# Visualize correlation
plt.figure(figsize=(6, 4))
sns.heatmap(correlation, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Between Arrival Delay & Departure Delay")
plt.show()


In [ ]:
df_new

# **PyCaret**

In [ ]:
!pip uninstall -y pycaret scikit-plot scipy
!pip install --no-cache-dir scipy==1.9.3
!pip install --no-cache-dir scikit-plot
!pip install --no-cache-dir pycaret

In [ ]:
import pandas as pd
from pycaret.regression import *

# Load the dataset
#file_path = "/content/df_new.csv"  # Adjust the file path if needed
df = df.copy()

# Define the target and features
target = "Arrival Delay (Minutes)"
features = ["Day", "Month", "Year", "Taxi-Out time (Minutes)", "Taxi-In time (Minutes)", "Scheduled Elapsed Time (Minutes)",
            "Origin Airport", "Carrier Code", "Destination Airport",
            "Scheduled departure hour", "Departure delay (Minutes)"]

# Select only the required columns
df = df[features + [target]]

# Initialize PyCaret for regression
exp_reg = setup(data=df, target=target, session_id=123,
    normalize=True, categorical_features=["Origin Airport", "Carrier Code", "Destination Airport"],encoding_method="onehot")


# Save the best model
best_model = compare_models()
save_model(best_model, "best_regression_model")

print("\nBest model saved as 'best_regression_model'.")


In [ ]:
print(best_model)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Compute correlation
correlation = df_new[['Arrival Delay (Minutes)','Origin Airport', 'Destination Airport']].corr()
print(correlation)

# Visualize correlation
plt.figure(figsize=(6, 4))
sns.heatmap(correlation, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Between Origin Airpor & Destination Airport")
plt.show()

In [ ]:
from pycaret.regression import load_model, pull, get_config, predict_model
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd

# Load saved model and pipeline
model = load_model("best_regression_model")

target = "Arrival Delay (Minutes)"
features = ["Day", "Month", "Year", "Taxi-Out time (Minutes)", "Taxi-In time (Minutes)", "Scheduled Elapsed Time (Minutes)",
            "Origin Airport", "Carrier Code", "Destination Airport",
            "Scheduled departure hour", "Departure delay (Minutes)"]

X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Predict on training data
train_preds_df = predict_model(model, data=X_train)
train_preds = train_preds_df.iloc[:, -1].values  # Last column is prediction

# Predict on test data
test_preds_df = predict_model(model, data=X_test)
test_preds = test_preds_df.iloc[:, -1].values

# Compute metrics
train_mae = mean_absolute_error(y_train, train_preds)
test_mae = mean_absolute_error(y_test, test_preds)

train_mse = mean_squared_error(y_train, train_preds)
test_mse = mean_squared_error(y_test, test_preds)

train_r2 = r2_score(y_train, train_preds)
test_r2 = r2_score(y_test, test_preds)

# Output results
print(f"📊 PyCaret Model Performance:")
print(f"   🔹 Train MAE: {train_mae:.2f} min")
print(f"   🔹 Test MAE: {test_mae:.2f} min")
print(f"   🔹 Train MSE: {train_mse:.2f}")
print(f"   🔹 Test MSE: {test_mse:.2f}")
print(f"   🔹 Train R²: {train_r2:.2f}")
print(f"   🔹 Test R²: {test_r2:.2f}")


# **Deep Learning Model**

In [ ]:
import pandas as pd
import numpy as np
import joblib
import xgboost as xgb
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.layers import LeakyReLU

# Define Features & Target
target = "Arrival Delay (Minutes)"
features = ["Day", "Month", "Year", "Taxi-Out time (Minutes)", "Taxi-In time (Minutes)",
            "Origin Airport", "Carrier Code", "Destination Airport",
            "Scheduled Elapsed Time (Minutes)",
            "Scheduled departure hour", "Departure delay (Minutes)"]

df = df[features + [target]].dropna()  # Drop missing values

# ✅ One-Hot Encode Categorical Features
categorical_features = ["Origin Airport", "Carrier Code", "Destination Airport"]
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
encoded_cats = encoder.fit_transform(df[categorical_features])
joblib.dump(encoder, "encoder.pkl")

# ✅ Standardize Numerical Features
numerical_features = ["Day", "Month", "Year", "Taxi-Out time (Minutes)", "Taxi-In time (Minutes)",
                      "Scheduled Elapsed Time (Minutes)",
                      "Scheduled departure hour", "Departure delay (Minutes)"]

scaler = StandardScaler()
scaled_numerics = scaler.fit_transform(df[numerical_features])
joblib.dump(scaler, "scaler.pkl")

# ✅ Combine Features
X = np.hstack((scaled_numerics, encoded_cats))
y = df[target].values

print(f"📊 Final feature count: {X.shape[1]}")  # Debugging feature count

# ✅ Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Train MLP (Neural Network) with improvements
mlp_model = Sequential([
    Dense(256, kernel_initializer=HeNormal(), input_shape=(X_train.shape[1],)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.2),  # Reduced from 0.3

    Dense(128, kernel_initializer=HeNormal()),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.15),  # Reduced from 0.2

    Dense(64, kernel_initializer=HeNormal()),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),

    Dense(32, kernel_initializer=HeNormal()),
    LeakyReLU(alpha=0.1),

    Dense(1)  # Output Layer
])

# Compile with MAE loss and lower LR
mlp_model.compile(optimizer=AdamW(learning_rate=0.001, weight_decay=1e-5),
                  loss="mae",  # Optimize MAE directly
                  metrics=["mae", "mse"])

# ✅ Callbacks for improved convergence
callbacks = [
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-5, verbose=1)
]

# ✅ Train longer with smart control
history = mlp_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,  # Longer training
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

# Save Keras model in a compatible format
mlp_model.save("mlp_model.keras")
# ✅ Evaluate Models
xgb_pred = xgb_model.predict(X_test)
mlp_pred = mlp_model.predict(X_test).flatten()

xgb_mae = mean_absolute_error(y_test, xgb_pred)
mlp_mae = mean_absolute_error(y_test, mlp_pred)

xgb_r2 = r2_score(y_test, xgb_pred)
mlp_r2 = r2_score(y_test, mlp_pred)

print(f"📉 XGBoost MAE: {xgb_mae:.2f}, R²: {xgb_r2:.2f}")
print(f"📉 MLP Neural Network MAE: {mlp_mae:.2f}, R²: {mlp_r2:.2f}")

# ✅ 95% Confidence Interval for MLP
residuals = y_test - mlp_pred
residual_std = np.std(residuals)
ci_upper = mlp_pred + 1.96 * residual_std
ci_lower = mlp_pred - 1.96 * residual_std

print(f"📉 Residual Std Dev: {residual_std:.2f}")
print(f"📉 95% Confidence Interval Range: ±{1.96 * residual_std:.2f} min")

# ✅ Plot Training Loss & Validation Loss
plt.figure(figsize=(12, 5))

# Loss Plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss', marker='o')
plt.plot(history.history['val_loss'], label='Validation Loss', marker='o')
plt.title('MLP Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.legend()

# R² Comparison Plot
plt.subplot(1, 2, 2)
models = ['XGBoost', 'MLP']
r2_scores = [xgb_r2, mlp_r2]
plt.bar(models, r2_scores, color=['blue', 'green'])
plt.title('Model R² Comparison')
plt.ylabel('R² Score')

plt.tight_layout()
plt.show()

# ✅ Plot Prediction with 95% Confidence Interval
plt.figure(figsize=(10, 6))
plt.plot(y_test, label='Actual', alpha=0.7)
plt.plot(mlp_pred, label='Predicted', alpha=0.7)
plt.fill_between(np.arange(len(y_test)), ci_lower, ci_upper, color='orange', alpha=0.3, label='95% CI')
plt.title('MLP Predictions with 95% Confidence Interval')
plt.xlabel('Sample Index')
plt.ylabel('Arrival Delay (Minutes)')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Updated MAPE function to handle zero values safely
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0  # Ignore zero values in y_true
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if np.any(mask) else np.inf

# ✅ Predictions on Training & Test Data
xgb_train_pred = xgb_model.predict(X_train)
xgb_test_pred = xgb_model.predict(X_test)

mlp_train_pred = mlp_model.predict(X_train).flatten()
mlp_test_pred = mlp_model.predict(X_test).flatten()

# ✅ Calculate MAE (Mean Absolute Error)
xgb_train_mae = mean_absolute_error(y_train, xgb_train_pred)
xgb_test_mae = mean_absolute_error(y_test, xgb_test_pred)

mlp_train_mae = mean_absolute_error(y_train, mlp_train_pred)
mlp_test_mae = mean_absolute_error(y_test, mlp_test_pred)

# ✅ Calculate R² Scores
xgb_train_r2 = r2_score(y_train, xgb_train_pred)
xgb_test_r2 = r2_score(y_test, xgb_test_pred)

mlp_train_r2 = r2_score(y_train, mlp_train_pred)
mlp_test_r2 = r2_score(y_test, mlp_test_pred)

# ✅ Calculate MAPE (Mean Absolute Percentage Error) with zero handling
xgb_train_mape = mean_absolute_percentage_error(y_train, xgb_train_pred)
xgb_test_mape = mean_absolute_percentage_error(y_test, xgb_test_pred)

mlp_train_mape = mean_absolute_percentage_error(y_train, mlp_train_pred)
mlp_test_mape = mean_absolute_percentage_error(y_test, mlp_test_pred)

# ✅ Print the Results
print(f"📊 **XGBoost Performance:**")
print(f"   - Train MAE: {xgb_train_mae:.2f} min")
print(f"   - Test MAE: {xgb_test_mae:.2f} min")
print(f"   - Train R²: {xgb_train_r2:.2f}")
print(f"   - Test R²: {xgb_test_r2:.2f}")
print(f"   - Train MAPE: {xgb_train_mape:.2f} %")
print(f"   - Test MAPE: {xgb_test_mape:.2f} %")

print(f"\n📊 **MLP Neural Network Performance:**")
print(f"   - Train MAE: {mlp_train_mae:.2f} min")
print(f"   - Test MAE: {mlp_test_mae:.2f} min")
print(f"   - Train R²: {mlp_train_r2:.2f}")
print(f"   - Test R²: {mlp_test_r2:.2f}")
print(f"   - Train MAPE: {mlp_train_mape:.2f} %")
print(f"   - Test MAPE: {mlp_test_mape:.2f} %")